In [4]:
# !pip3 install --upgrade torch --user
# !pip3 install torchvision --user

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import pickle
import tqdm

import torch
import torch.nn.functional as F
import torch.nn as nn


from torchvision import datasets
from torchvision import transforms

# !pip3 install ipywidgets --user

In [2]:
event_step = 30720
ada_step = 12
collection_step = 960
readout_step = 800

In [3]:
def load_files(path_clear='/data/martemev/output_data/',
               path_noise='/data/martemev/input_noised_data/'):
    for index in range(0, 100):
        clear_data = np.load(os.path.join(path_clear, 'onlysignal_evt{}.npy'.format(index+1)))[:, 2:]
        noised_data = np.load(os.path.join(path_noise, 'event_{}_noised.npy'.format(index)))[:, 3:]
        yield torch.Tensor(clear_data), torch.Tensor(noised_data)

In [4]:
def load_file(index, path_clear='/data/martemev/output_data/',
               path_noise='/data/martemev/input_noised_data/'):
    clear_data = np.load(os.path.join(path_clear, 'onlysignal_evt{}.npy'.format(index+1)))[:, 2:]
    noised_data = np.load(os.path.join(path_noise, 'event_{}_noised.npy'.format(index)))[:, 3:]
    return torch.Tensor(clear_data), torch.Tensor(noised_data)

In [5]:
# right now we are only returning first collection plane
signal_panes = [1, 2, 5, 6, 9, 10]

def normalize(im):
    return (im - im.min())/(im.max()-im.min())

def get_planes(clear_file, noised_file):
    for index in signal_panes:
        clear_plane = clear_file[index*(readout_step*2 + collection_step):
                                  (index+1)*(readout_step*2 + collection_step)]
        noised_plane = noised_file[index*(readout_step*2 + collection_step):
                                            (index+1)*(readout_step*2 + collection_step)]
        if clear_plane[2*readout_step:].max() == 0:
            continue
        yield clear_plane[2*readout_step:], \
            noised_plane[2*readout_step:]
#         yield normalize(clear_plane[2*readout_step:]), \
#             normalize(noised_plane[2*readout_step:])
        

In [6]:
def get_crop(clear_plane, noised_plane, total_crops=1000, crop_shape=(32, 32)):
    probs = normalize(clear_plane)
    probs[probs == 0] += probs.mean()
    distr = torch.distributions.binomial.Binomial(total_count=5, probs=probs)
    x, y = clear_plane.shape
    c_x, c_y = crop_shape[0]//2, crop_shape[1]//2

    for i in tqdm.tqdm_notebook(range(total_crops), desc='crops', leave=False):
        samples = torch.nonzero(distr.sample())
        while len(samples) < 1:
            samples = torch.nonzero(distr.sample())
        sample = np.random.choice(len(samples))
        sample = samples[sample]

        sample = (min(max(int(sample[0]), c_x), x-c_x), min(max(int(sample[1]), c_y), y-c_y))
        clear_crop = clear_plane[sample[0]-c_x:sample[0]+c_x, sample[1]-c_y:sample[1]+c_y]
        noised_crop = noised_plane[sample[0]-c_x:sample[0]+c_x, sample[1]-c_y:sample[1]+c_y]
        yield clear_crop.unsqueeze(0), noised_crop.unsqueeze(0)


In [7]:
from random import random

In [8]:
clear_dataset_train = []
noised_dataset_train = []
clear_dataset_val = []
noised_dataset_val = []

for file_clear, file_noise in tqdm.tqdm_notebook(load_files(), total=100, desc='files'):
    if random() > 0.9:
        for plane_clear, plane_noise in tqdm.tqdm_notebook(get_planes(file_clear, file_noise), desc='train planes', leave=False):
            clear_dataset_val.append(plane_clear)
            noised_dataset_val.append(plane_noise)
    else:
        for plane_clear, plane_noise in tqdm.tqdm_notebook(get_planes(file_clear, file_noise), desc='val planes', leave=False):
            clear_dataset_train.append(plane_clear)
            noised_dataset_train.append(plane_noise)
        
        

In [9]:
noised_data_train = torch.stack(noised_dataset_train, dim=0)
clear_data_train = torch.stack(clear_dataset_train, dim=0)

torch.save(noised_data_train, '../Data/noised_events_collection_unnormalized_train.tensor')
torch.save(clear_data_train, '../Data/clear_events_collection_unnormalized_train.tensor')

noised_data_val = torch.stack(noised_dataset_val, dim=0)
clear_data_val = torch.stack(clear_dataset_val, dim=0)

torch.save(noised_data_val, '../Data/noised_events_collection_unnormalized_val.tensor')
torch.save(clear_data_val, '../Data/clear_events_collection_unnormalized_val.tensor')

### Here we saved all needed planes

Now let's crop'em

In [5]:
noised_data_train = torch.load('../Data/noised_events_collection_unnormalized_train.tensor')
clear_data_train = torch.load('../Data/clear_events_collection_unnormalized_train.tensor')


noised_data_val = torch.load('../Data/noised_events_collection_unnormalized_val.tensor')
clear_data_val = torch.load('../Data/clear_events_collection_unnormalized_val.tensor')


In [6]:
clear_data_train.shape

torch.Size([156, 960, 4492])

In [9]:
clear_crops_train = []
noised_crops_train = []

for clear_plane, noised_plane in tqdm.tqdm_notebook(zip(clear_data_train, noised_data_train), total=len(clear_data_train), desc='Planes'):
    for clear_crop, noised_crop in get_crop(clear_plane, noised_plane, 500, (32, 32)):
        clear_crops_train.append(clear_crop)
        noised_crops_train.append(noised_crop)
        

In [57]:
# for i in range(100):
#     plt.imshow(clear_crops_[i].numpy().reshape(64, 64))
#     plt.show()

In [11]:
torch.save(torch.stack(clear_crops_train, dim=0), '../Data/clear_crops.32.train.tensor')
torch.save(torch.stack(noised_crops_train, dim=0), '../Data/noised_crops.32.train.tensor')

In [13]:
clear_crops_val = []
noised_crops_val = []

for clear_plane, noised_plane in tqdm.tqdm_notebook(zip(clear_data_val, noised_data_val), total=len(clear_data_val), desc='Planes'):
    for clear_crop, noised_crop in get_crop(clear_plane, noised_plane, 500, (32, 32)):
        clear_crops_val.append(clear_crop)
        noised_crops_val.append(noised_crop)
        

In [15]:
torch.save(torch.stack(clear_crops_val, dim=0), '../Data/clear_crops.32.val.tensor')
torch.save(torch.stack(noised_crops_val, dim=0), '../Data/noised_crops.32.val.tensor')